In [1]:
import numpy as np 
import pandas as pd
from pandas import DataFrame as df
import datetime
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import folium
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import rc
rc('font', family='AppleGothic')
import plotly.express as px
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from IPython.display import IFrame

In [2]:
master_data = pd.read_stata("masterfile_no_outlier.dta")
master_data['date'] = pd.to_datetime(master_data['REPORTINGMONTH'])
master_data['year'] = master_data['date'].dt.year
master_data['month'] = master_data['date'].dt.month
master_data['yyyymm'] = master_data['year']*100 + master_data['month']

In [3]:
listing_type = ['Loft', 'Entire apartment','Entire house','Place','Apartment','Condominium','In-law','Serviced apartment','Entire condominium','Townhouse','Entire bed & breakfast','House']
rev900_type = master_data.query("PROPERTYTYPE == @listing_type")

In [4]:
def calculator(data):
    d = {}
    d['OCCUPANCYRATE'] = data['OCCUPANCYRATE'].mean()  # Occupancy Rate
    d['REVENUEUSD'] = data['REVENUEUSD'].mean()
    d['ADRUSD'] = data['ADRUSD'].mean()
    d['RESERVATIONDAYS'] = data['RESERVATIONDAYS'].mean()
    d['AVAILABLEDAYS'] = data['AVAILABLEDAYS'].mean()
    d['BLOCKEDDAYS'] = data['BLOCKEDDAYS'].mean()
    d['count'] = data['PROPERTYID'].count()
    return pd.Series(d, index=['OCCUPANCYRATE', 'REVENUEUSD', 'ADRUSD', 'RESERVATIONDAYS', 'AVAILABLEDAYS', 'BLOCKEDDAYS', 'count'])

In [5]:
def map_source(gu):
    gu_source = rev900_type[(rev900_type['gu']==str(gu))]
    gu_listing = gu_source.groupby(['PROPERTYID']).apply(calculator)
    gu_listing_over10 = gu_listing.query('count >= 10')
    print(gu_listing_over10.describe())
    gu_1000dollar = gu_listing_over10.query("REVENUEUSD >= 1000")

    gu_target_list = list(gu_listing_over10.index)
    gu_target_panel = gu_source.query("PROPERTYID == @gu_target_list")
    gu_1000dollar_list = list(gu_1000dollar.index)
    gu_1000dollar_panel = gu_source.query("PROPERTYID == @gu_1000dollar_list")
    gu_target_unique = gu_target_panel.drop_duplicates(['PROPERTYID'])
    gu_1000dollar_unique = gu_1000dollar_panel.drop_duplicates(['PROPERTYID'])
    return gu_target_unique, gu_1000dollar_unique

def map_generator(source):
    map_target = folium.Map(location=[37.538801, 127.004746], tiles='cartodbpositron')
    Lat = list(source['LATITUDE'])
    Longt = list(source['LONGITUDE'])
    for i in range(len(Lat)):
        folium.CircleMarker([Lat[i],Longt[i]],radius=0.5,color='#335EFF',fill_color='#335EFF').add_to(map_target)
    return map_target

* * *

<pre>
panel에 적어도 10번 이상, 즉 10달 이상 꾸준히 performance 성과를 데이터로 보여주는 listing으로 데이터 범위를 좁혔음.
</pre>

# Data 범위
<pre>
1. 평균 수익이 900달러를 넘는 숙소 형태(Loft, Apt, House..)
2. 10달 이상 꾸준히 영업을 한 숙소
3. 강남구/마포구/용산구/종로구/중구 
로 데이터 범위를 좁힌 후, 
1,2,3을 만족하는 listing을 지도 위에 plot --> 어떤 건물 형태에 주로 분포하는지 확인 

+

4. 월 평균 1000달러 이상의 수익을 낸 숙소 
조건까지 부여하여 1,2,3,4를 모두 만족하는 숙소 listing을 지도 위에 plot --> 수익성이 좋은 listing의 분포에 집중

</pre>

# 1. 강남구

In [6]:
total, dollar = map_source("강남구")

       OCCUPANCYRATE   REVENUEUSD      ADRUSD  RESERVATIONDAYS  AVAILABLEDAYS  \
count     859.000000   859.000000  859.000000       859.000000     859.000000   
mean        0.613088  1132.368043   77.343921        15.568003      10.076418   
std         0.204672   543.140939   38.946012         5.652105       5.934502   
min         0.069600    60.000000   19.074615         2.100000       0.166667   
25%         0.471991   756.863800   57.734974        11.275253       5.581250   
50%         0.640478  1093.909058   69.590912        16.076923       9.083333   
75%         0.768381  1448.553162   83.164333        19.717143      13.435606   
max         0.994583  3669.487305  288.386658        29.000000      28.100000   

       BLOCKEDDAYS       count  
count   859.000000  859.000000  
mean      4.794448   16.845169  
std       3.362940    6.854499  
min       0.000000   10.000000  
25%       2.219807   12.000000  
50%       4.150000   15.000000  
75%       6.975000   20.000000  
max   

<pre>
강남구에 위의 1,2,3의 조건을 만족하는 숙소는 859개 존재.

평균 77달러의 숙박료로, occupancy rate은 61% --> 수익은 1132달러
Reservationdays + Availabledays + Blockeddays가 평균적으로 30에 달하는 것을 보아 투자 수단으로 사용되는 것을 알 수 있음 

평균 영업 기간은 16개월임
</pre>

### 강남구 property type:  rev900+ plot

In [7]:
map_generator(total).save("gangnam_total.html")
IFrame(src='gangnam_total.html', width=700, height=600)

### 강남구 listing: rev 1000+ plot

In [8]:
map_generator(dollar).save("gangnam_1000.html")
IFrame(src='gangnam_1000.html', width=700, height=600)

<pre>
강남구 월 1000달러 이상 꾸준히 수익 낸 listing
- 강남역 1번 출구 방면 오피스텔 밀집 지역 (고메램 맞은편)
- 신논현역 ~ 신사역 사이 다가구 지역 
- 신사 가로수길 다가구 지역 
- 역삼 GS타워 뒷편 다가구 밀집지역
- 선릉역 오피스텔 지역

apt라는 property type으로 분류 되었지만, 
분포 지역 살펴보면 다가구와 오피스텔 위주임
</pre>

* * *

# 2. 마포구

In [9]:
total, dollar = map_source("마포구")

       OCCUPANCYRATE   REVENUEUSD       ADRUSD  RESERVATIONDAYS  \
count    1796.000000  1796.000000  1796.000000      1796.000000   
mean        0.639227  1216.312049    71.536070        17.177486   
std         0.195091   642.235582    32.793731         5.647072   
min         0.061600    77.380951    13.445384         1.800000   
25%         0.510712   706.550552    51.811248        13.216667   
50%         0.682321  1208.195618    69.151608        17.960294   
75%         0.787600  1604.647491    82.277382        21.636364   
max         0.994091  3496.078125   273.165009        29.000000   

       AVAILABLEDAYS  BLOCKEDDAYS        count  
count    1796.000000  1796.000000  1796.000000  
mean        9.792603     3.455091    17.596325  
std         5.642661     2.930690     7.122655  
min         0.181818     0.000000    10.000000  
25%         5.597059     1.387821    12.000000  
50%         8.500000     2.646087    15.000000  
75%        13.143939     4.636364    21.000000  
max 

<pre>
마포구에 위의 1,2,3의 조건을 만족하는 숙소는 1796개 존재.

평균 71달러의 숙박료로, occupancy rate은 64% --> 수익은 1216달러
Reservationdays + Availabledays + Blockeddays가 평균적으로 30에 달하는 것을 보아 투자 수단으로 사용되는 것을 알 수 있음 

평균 영업 기간은 17개월임
</pre>

### 마포구 property type:  rev900+ plot

In [10]:
map_generator(total).save("mapo_total.html")
IFrame(src='mapo_total.html', width=700, height=600)

### 마포구 listing: rev 1000+ plot

In [11]:
map_generator(dollar).save("mapo_1000.html")
IFrame(src='mapo_1000.html', width=700, height=600)

<pre>
마포구 월 1000달러 이상 꾸준히 수익 낸 listing
- 공덕역/마포역 오피스텔 
- 합정역 오피스텔 
- 홍대~신촌 다가구 
- 연남동 다가구

apt라는 property type으로 분류 되었지만, 
분포 지역 살펴보면 다가구와 오피스텔 위주임
</pre>

* * *

# 3. 용산구

In [12]:
total, dollar = map_source("용산구")

       OCCUPANCYRATE   REVENUEUSD      ADRUSD  RESERVATIONDAYS  AVAILABLEDAYS  \
count     912.000000   912.000000  912.000000       912.000000     912.000000   
mean        0.590739  1127.816957   75.412997        15.717307      11.092770   
std         0.201063   585.113770   38.771581         5.616831       5.862751   
min         0.076700    46.900002   14.421110         2.300000       0.500000   
25%         0.432767   717.142868   51.736333        10.880952       6.500000   
50%         0.619725  1057.230652   68.013748        16.262963      10.108140   
75%         0.756742  1477.691376   89.106461        20.331933      15.060185   
max         0.973842  3342.953369  257.035309        28.666667      27.812500   

       BLOCKEDDAYS       count  
count   912.000000  912.000000  
mean      3.614998   18.228070  
std       2.957310    7.989191  
min       0.000000   10.000000  
25%       1.500000   12.000000  
50%       2.950957   16.000000  
75%       4.912587   21.000000  
max   

<pre>
용산구에 위의 1,2,3의 조건을 만족하는 숙소는 912개 존재.

평균 75달러의 숙박료로, occupancy rate은 59% --> 수익은 1127달러
Reservationdays + Availabledays + Blockeddays가 평균적으로 30에 달하는 것을 보아 투자 수단으로 사용되는 것을 알 수 있음 

평균 영업 기간은 18개월임
</pre>

In [13]:
map_generator(total).save("yongsan_total.html")
IFrame(src='yongsan_total.html', width=700, height=600)

In [14]:
map_generator(dollar).save("yongsan_1000.html")
IFrame(src='yongsan_1000.html', width=700, height=600)

<pre>
용산구 월 1000달러 이상 꾸준히 수익 낸 listing
- 서울역 서부역 방면 오피스텔 밀집 지역
- 남영역 오피스텔 밀집지역 
- 삼각지역 오피스텔 밀집지역 
- 신용산역 뒷편 다가구 밀집지역 
- 이태원 다가구 밀집지역

apt라는 property type으로 분류 되었지만, 
분포 지역 살펴보면 다가구와 오피스텔 위주임
</pre>

* * *

# 4. 종로구

In [15]:
total, dollar = map_source("종로구")

       OCCUPANCYRATE   REVENUEUSD      ADRUSD  RESERVATIONDAYS  AVAILABLEDAYS  \
count     647.000000   647.000000  647.000000       647.000000     647.000000   
mean        0.563865  1130.783316   80.632825        15.124436      11.855791   
std         0.217162   614.708034   45.822514         6.225318       6.244236   
min         0.079571   142.199997   15.344546         2.000000       0.750000   
25%         0.399200   646.729584   52.535555        10.227273       7.000000   
50%         0.594539  1089.125000   68.319229        15.388889      10.866667   
75%         0.738929  1481.702393   95.078224        20.108824      16.225000   
max         0.969437  3522.899902  294.259247        28.279070      27.142857   

       BLOCKEDDAYS       count  
count   647.000000  647.000000  
mean      3.452305   17.761978  
std       2.789677    7.185657  
min       0.000000   10.000000  
25%       1.424812   12.000000  
50%       2.857143   16.000000  
75%       4.847490   21.000000  
max   

<pre>
종로구에 위의 1,2,3의 조건을 만족하는 숙소는 647개 존재.

평균 80달러의 숙박료로, occupancy rate은 56% --> 수익은 1130달러
Reservationdays + Availabledays + Blockeddays가 평균적으로 30에 달하는 것을 보아 투자 수단으로 사용되는 것을 알 수 있음 

평균 영업 기간은 17개월임
</pre>

In [16]:
map_generator(total).save("jongro_total.html")
IFrame(src='jongro_total.html', width=700, height=600)

In [17]:
map_generator(dollar).save("jongro_1000.html")
IFrame(src='jongro_1000.html', width=700, height=600)

# 5. 중구

In [18]:
total, dollar = map_source("중구")

       OCCUPANCYRATE   REVENUEUSD       ADRUSD  RESERVATIONDAYS  \
count    1076.000000  1076.000000  1076.000000      1076.000000   
mean        0.655068  1356.160846    79.110575        17.603591   
std         0.175253   610.843797    32.326571         5.254476   
min         0.138727   157.769226    13.493077         3.000000   
25%         0.545643   905.643753    60.109451        13.755348   
50%         0.690769  1315.363159    73.554623        18.779221   
75%         0.784673  1718.584381    90.020506        21.802381   
max         0.958200  3676.811035   264.250671        28.225806   

       AVAILABLEDAYS  BLOCKEDDAYS        count  
count    1076.000000  1076.000000  1076.000000  
mean        9.335506     3.484076    19.221190  
std         4.984613     3.128394     8.162498  
min         0.700000     0.000000    10.000000  
25%         5.569196     1.250000    13.000000  
50%         8.357143     2.642857    17.000000  
75%        12.090787     4.651961    24.000000  
max 

<pre>
중구에 위의 1,2,3의 조건을 만족하는 숙소는 1076개 존재.

평균 79달러의 숙박료로, occupancy rate은 65% --> 수익은 1356달러
Reservationdays + Availabledays + Blockeddays가 평균적으로 30에 달하는 것을 보아 투자 수단으로 사용되는 것을 알 수 있음 

평균 영업 기간은 19개월임
</pre>

In [19]:
map_generator(total).save("junggu_total.html")
IFrame(src='junggu_total.html', width=700, height=600)

In [20]:
map_generator(dollar).save("junggu_1000.html")
IFrame(src='junggu_1000.html', width=700, height=600)